In [6]:
!pip install llama-index --quiet
!pip install llama-index-embeddings-huggingface --quiet
!pip install llama-index-llms-huggingface --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00


In [22]:
!pip install llama-index-embeddings-huggingface-api --quiet
!pip install llama-index-embeddings-openai --quiet

In [7]:
import os
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.evaluation import FaithfulnessEvaluator, RelevancyEvaluator

In [4]:
# Embedding Model
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding

In [14]:
# Define embedding models
embedding_models = {
    "openai": OpenAIEmbedding(model="text-embedding-3-large"),
    "bge": HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-large-en"),
    "jina": HuggingFaceInferenceAPIEmbedding(model_name="jinaai/jina-embeddings-v2-base-en")
}

In [9]:
# Define LLMs
llms = {
    "gpt-4": OpenAI(model="gpt-4o-mini"),

}

In [12]:
documents = SimpleDirectoryReader(input_files=['/content/data/Final Policy document_LICs New Jeevan Shanti_V05_logo.pdf']).load_data()

In [ ]:
# Iterate over embedding models for indexing
vector_indexes = {}
for name, embed_model in embedding_models.items():
    index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
    vector_indexes[name] = index

In [ ]:
# Evaluate retrieval and synthesis
for embed_name, index in vector_indexes.items():
    for llm_name, llm in llms.items():
        retriever = index.as_retriever()
        evaluator = FaithfulnessEvaluator()
        query = "Summarize the document contents."
        retrieved_docs = retriever.retrieve(query)
        response = llm.complete(query)
        score = evaluator.evaluate(response, retrieved_docs)
        print(f"Embedding: {embed_name}, LLM: {llm_name}, Faithfulness Score: {score}")